In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
from scipy import stats

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

from transformers import BertTokenizer, BertModel
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [2]:
train_df = pd.read_csv('clean_data.csv')
val_df = pd.read_csv('clean_val.csv')
test_df = pd.read_csv('clean_test.csv')

In [3]:
train_df.dropna(inplace = True)

In [4]:
with open("numerical.txt", "r", encoding="utf-8") as file:
    numeric_features = [line.strip() for line in file.readlines()]
    
with open("cat.txt", "r", encoding="utf-8") as file:
    cat_features = [line.strip() for line in file.readlines()]
    
with open("text.txt", "r", encoding="utf-8") as file:
    text_features = [line.strip() for line in file.readlines()]

In [5]:
print('Total number of Numerical Features and one target:', len(numeric_features))
print('Total number of Categorical Features:',len(cat_features))
print('Total number of Text Features:',len(text_features))

Total number of Numerical Features and one target: 9
Total number of Categorical Features: 70
Total number of Text Features: 2


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
embedding_model = BertModel.from_pretrained('bert-base-multilingual-cased')
embedding_model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [7]:
class BoxCoxTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lambdas_ = {}

    def fit(self, X, y=None):
        X = X.copy()
        for column in X.columns:
            if any(X[column] <= 0):
                X[column] = X[column] + abs(X[column].min()) + 1
            _, lambda_optimal = stats.boxcox(X[column])
            self.lambdas_[column] = lambda_optimal
        return self

    def transform(self, X):
        X = X.copy()
        for column, lambda_optimal in self.lambdas_.items():
            if any(X[column] <= 0):
                X[column] = X[column] + abs(X[column].min()) + 1
            X[column] = stats.boxcox(X[column], lmbda=lambda_optimal)
        return X

def get_bert_embeddings(text_series, batch_size):
    embeddings = []
    embedding_model.eval()  
    
    for i in range(0, len(text_series), batch_size):
        batch_texts = text_series[i:i + batch_size]
        
        if not batch_texts: 
            continue
        
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = embedding_model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_embeddings)
    
    if embeddings:
        return np.vstack(embeddings)
    else:
        return np.array([])
    
def concat(df, text_features, batch_size, pooling, is_train = False):

    embeddings_list = []

    for feature in text_features:
        print(feature)
        text_embeddings = get_bert_embeddings(df[feature].tolist(), batch_size)

        if pooling == 'max':
            text_embeddings = np.max(text_embeddings, axis=1)  
        elif pooling == 'mean':
            text_embeddings = np.mean(text_embeddings, axis=1)  

        if text_embeddings.ndim == 1:
            text_embeddings = text_embeddings.reshape(-1, 1)

        embeddings_list.append(text_embeddings)
        
    if is_train:
        transformed_features = preprocessor.fit_transform(df).toarray()
    else:
        transformed_features = preprocessor.transform(df).toarray()

    combined_features = np.concatenate(embeddings_list + [transformed_features], axis=1)

    return combined_features

In [8]:
class MyDataset(Dataset):
    def __init__(self, combined_features, norm_target, target):
        self.combined_features = np.array(combined_features)
        self.norm_target = np.array(norm_target)
        self.target = np.array(target)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        combined_features = self.combined_features[index]
        norm_target = self.norm_target[index] 
        target = self.target[index] 
        
        return {'combined_features': torch.tensor(combined_features, dtype=torch.float32),
                'norm_target':torch.tensor(norm_target, dtype=torch.float32),
                'target':torch.tensor(target, dtype=torch.float32)
        }

In [9]:
def normalize(x):
    return (x - train_df['price'].min()) / (train_df['price'].max() - train_df['price'].min())

def denormalize(x):
    return x * (train_df['price'].max() - train_df['price'].min()) + train_df['price'].min()

In [10]:
train_df['normalized_price'] = train_df['price'].apply(normalize)
val_df['normalized_price'] = val_df['price'].apply(normalize)
test_df['normalized_price'] = test_df['price'].apply(normalize)

In [11]:
numeric_transformer = Pipeline([
    ('boxcox', BoxCoxTransformer()),
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

In [12]:
train_combined_features = concat(train_df, text_features, batch_size = 32, pooling = 'max', is_train = True)
val_combined_features = concat(val_df, text_features, batch_size = 32, pooling = 'max', is_train = False)
test_combined_features = concat(test_df, text_features, batch_size = 32, pooling = 'max', is_train = False)

summary
host_about
summary
host_about
summary
host_about


In [13]:
batch_size = 32

train_norm_y = train_df['normalized_price']
train_y = train_df['price']

val_norm_y = val_df['normalized_price']
val_y = val_df['price']

test_norm_y = test_df['normalized_price']
test_y = test_df['price']

train_dataset = MyDataset(train_combined_features, train_norm_y, train_y)
val_dataset = MyDataset(val_combined_features, val_norm_y, val_y)
test_dataset = MyDataset(test_combined_features, test_norm_y, test_y)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [14]:
class MLPRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(MLPRegressionModel, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 64)
        self.bn1 = nn.BatchNorm1d(64)
        
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        
        self.fc3 = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        
        x = self.fc1(x)
        x = self.bn1(x)        
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.bn2(x)        
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

In [15]:
input_dim = train_combined_features.shape[1]
model = MLPRegressionModel(input_dim=input_dim)
model = model.to(device)

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [17]:
from tqdm import tqdm

def rmse(y, y_predict):
    mse = mean_squared_error(y, y_predict)
    return math.sqrt(mse)

def train(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    start_time = time.time()
    
    for batch in tqdm(data_loader, desc="Training", unit="batch"):
        inputs = batch['combined_features'].to(device)
        norm_target = batch['norm_target'].to(device)
        targets = batch['target'].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, norm_target.unsqueeze(1))
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        outputs = denormalize(outputs)
        
        all_preds.extend(outputs.cpu().tolist())
        all_labels.extend(targets.cpu().tolist())
        
    avg_loss = total_loss/len(data_loader)
    train_mae = mean_absolute_error(all_labels, all_preds)
    train_rmse = rmse(all_labels, all_preds)

    end_time = time.time()  
    epoch_duration = end_time - start_time  
    
    print(f'Training Loss: {avg_loss:.4f}, Train MAE: {train_mae:.4f}, Train RMSE: {train_rmse:.4f}, Time: {epoch_duration:.2f} seconds')
    return avg_loss, train_mae

def evaluate(model, data_loader, criterion, device):
    
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validation", unit="batch"):
            
            inputs = batch['combined_features'].to(device)
            norm_target = batch['norm_target'].to(device)
            targets = batch['target'].to(device)
            
            outputs = model(inputs)
            
            loss = criterion(outputs, norm_target.unsqueeze(1))
            total_loss += loss.item()
            
            outputs = denormalize(outputs)
            all_preds.extend(outputs.cpu().tolist())
            all_labels.extend(targets.cpu().tolist())
    

    avg_loss = total_loss/len(data_loader)
    val_mae = mean_absolute_error(all_labels, all_preds)
    val_rmse = rmse(all_labels, all_preds)

    end_time = time.time()  
    epoch_duration = end_time - start_time  
    
    print(f'Validation Loss: {avg_loss:.4f}, Validation MAE: {val_mae:.4f}, Validation RMSE: {val_rmse:.4f}, Time: {epoch_duration:.2f} seconds')
    
    return avg_loss, val_mae

In [18]:
best_val_loss = float('inf')  
best_val_mae = float('inf')              
model_save_path = "10_.pth"

num_epochs = 50

train_losses = []
val_losses = []
train_mae_scores = []
val_mae_scores = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    
    train_loss, train_mae = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_mae = evaluate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)  
    train_mae_scores.append(train_mae)  
    val_losses.append(val_loss)  
    val_mae_scores.append(val_mae)  
    
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        torch.save(model.state_dict(), model_save_path)   
        print(f"Best model saved with validation MAE: {best_val_mae:.4f}")

    print('-' * 100)

Epoch 1/50


Training: 100%|██████████████████████████████████████████████████████████████████| 561/561 [00:04<00:00, 116.29batch/s]


Training Loss: 0.0240, Train MAE: 81.8976, Train RMSE: 124.3496, Time: 4.86 seconds


Validation: 100%|██████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 582.31batch/s]


Validation Loss: 0.1214, Validation MAE: 56.3910, Validation RMSE: 280.7777, Time: 0.13 seconds
Best model saved with validation MAE: 56.3910
----------------------------------------------------------------------------------------------------
Epoch 2/50


...

Validation Loss: 0.1187, Validation MAE: 51.6365, Validation RMSE: 277.4666, Time: 0.14 seconds
----------------------------------------------------------------------------------------------------
Epoch 50/50


Training: 100%|██████████████████████████████████████████████████████████████████| 561/561 [00:03<00:00, 154.13batch/s]


Training Loss: 0.0048, Train MAE: 36.1257, Train RMSE: 55.7992, Time: 3.67 seconds


Validation: 100%|██████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 492.83batch/s]

Validation Loss: 0.1182, Validation MAE: 51.9002, Validation RMSE: 277.1071, Time: 0.15 seconds
----------------------------------------------------------------------------------------------------


In [19]:
model1 = MLPRegressionModel(input_dim=input_dim)
model1.to(device)

model1.load_state_dict(torch.load(model_save_path))
model1.to(device)
all_labels = []
all_preds = []

start_time = time.time()

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing", unit="batch"):

        inputs = batch['combined_features'].to(device)
        norm_target = batch['norm_target'].to(device)
        targets = batch['target'].to(device)

        outputs = model(inputs)

        loss = criterion(outputs, norm_target.unsqueeze(1))
        outputs = denormalize(outputs)
        all_preds.extend(outputs.cpu().tolist())
        all_labels.extend(targets.cpu().tolist())
        
all_labels = np.array(all_labels).flatten()
all_preds = np.array(all_preds).flatten()
        
test_mae = mean_absolute_error(all_labels, all_preds)
test_rmse = rmse(all_labels, all_preds)

end_time = time.time()  
epoch_duration = end_time - start_time

print(f'Testing MAE: {test_mae:.4f}, Testing RMSE: {test_rmse:.4f}, Time: {epoch_duration:.2f} seconds')

Testing: 100%|█████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 517.88batch/s]

Testing MAE: 45.8529, Testing RMSE: 137.4732, Time: 0.14 seconds
